# Entrenando un Modelo Markoviano Latente (HMM)

## Corpus de español: 

* AnCora | Github: https://github.com/UniversalDependencies/UD_Spanish-AnCora

* usamos el conllu parser para leer el corpus: https://pypi.org/project/conllu/

* Etiquetas Universal POS (Documentación): https://universaldependencies.org/u/pos/

In [1]:
#@title dependencias previas
!pip install conllu
!git clone https://github.com/UniversalDependencies/UD_Spanish-AnCora.git

fatal: la ruta de destino 'UD_Spanish-AnCora' ya existe y no es un directorio vacío.


In [2]:
#@title leyendo el corpus AnCora
# Función que permite la data en formato conllu
from conllu import parse_incr 
wordList = []
data_file = open("UD_Spanish-AnCora/es_ancora-ud-dev.conllu", "r", encoding="utf-8")
for tokenlist in parse_incr(data_file):
    print(tokenlist.serialize())

# sent_id = dev-s1
# text = El gobernante, con ganada fama desde que llegó hace 16 meses al poder de explotar al máximo su oratoria y acusado por sus detractores de incontinencia verbal, enmudeció desde el momento en el que el Tribunal Supremo de Justicia (TSJ) decidió suspender temporalmente los comicios múltiples ante la imposibilidad "técnica" de celebrarlos el 28 de mayo.
1	El	el	DET	DET	Definite=Def|Gender=Masc|Number=Sing|PronType=Art	2	det	_	_
2	gobernante	gobernante	NOUN	NOUN	Gender=Masc|Number=Sing	30	nsubj	_	SpaceAfter=No
3	,	,	PUNCT	PUNCT	PunctType=Comm	6	punct	_	_
4	con	con	ADP	ADP	AdpType=Prep	6	case	_	_
5	ganada	ganado	ADJ	ADJ	Gender=Fem|Number=Sing|VerbForm=Part	6	amod	_	_
6	fama	fama	NOUN	NOUN	Gender=Fem|Number=Sing	2	nmod	_	_
7	desde	desde	ADP	ADP	AdpType=Prep	9	mark	_	_
8	que	que	SCONJ	SCONJ	_	9	mark	_	_
9	llegó	llegar	VERB	VERB	Mood=Ind|Number=Sing|Person=3|Tense=Past|VerbForm=Fin	6	acl	_	_
10	hace	hacer	VERB	AUX	Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin	


# sent_id = dev-s280
# text = No la echaba de menos.
1	No	no	ADV	ADV	Polarity=Neg	3	advmod	_	_
2	la	él	PRON	PRON	Case=Acc|Gender=Fem|Number=Sing|Person=3|PrepCase=Npr|PronType=Prs	3	obj	_	_
3	echaba	echar	VERB	VERB	Mood=Ind|Number=Sing|Person=3|Tense=Imp|VerbForm=Fin	0	root	_	_
4	de	de	ADP	ADP	AdpType=Prep	5	case	_	_
5	menos	menos	ADV	ADV	Degree=Cmp	3	advmod	_	SpaceAfter=No
6	.	.	PUNCT	PUNCT	PunctType=Peri	3	punct	_	_


# sent_id = dev-s281
# text = A ratos estaba seguro de que lo sabía todo, porque su impasibidad, tan pura, no podía ser natural.
1	A	a	ADP	ADP	AdpType=Prep	4	advmod	_	MWE=A_ratos|MWEPOS=ADV
2	ratos	ratos	NOUN	NOUN	_	1	fixed	_	_
3	estaba	estar	AUX	AUX	Mood=Ind|Number=Sing|Person=3|Tense=Imp|VerbForm=Fin	4	cop	_	_
4	seguro	seguro	ADJ	ADJ	Gender=Masc|Number=Sing	0	root	_	_
5	de	de	ADP	ADP	AdpType=Prep	8	mark	_	_
6	que	que	SCONJ	SCONJ	_	8	mark	_	_
7	lo	él	PRON	PRON	Case=Acc|Gender=Masc|Number=Sing|Person=3|PrepCase=Npr|PronType=Prs	8	obj	_	_
8	sabía	saber	VERB	VERB	Mood=In

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [3]:
#@title Estructura de los tokens etiquetados del corpus
tokenlist[1]

{'id': 2,
 'form': 'cierto',
 'lemma': 'cierto',
 'upos': 'ADJ',
 'xpos': 'ADJ',
 'feats': {'Gender': 'Masc', 'Number': 'Sing'},
 'head': 3,
 'deprel': 'nsubj',
 'deps': None,
 'misc': None}

In [4]:
tokenlist[1]['form']+'|'+tokenlist[1]['upos']

'cierto|ADJ'

## Entrenamiento del modelo - Calculo de conteos:
Lo siguiente es un elemento diccionario que se refiere a cuantas veces aparece un ``tag``
* tags (tags) `tagCountDict`: $C(tag)$

Lo siguiente es un elemento diccionario que son las emisiones que se refiere a cuantas veces dado un ``tag`` le corresponde un ``word``
* emisiones (word|tag) `emissionProbDict`: $C(word|tag)$

Lo siguiente es un elemento diccionario que son las transiciones que se refiere a cuantas veces dado un ``tag`` previo le corresponde un ``tag`` en la posición siguiente
* transiciones (tag|prevtag) `transitionDict`: $C(tag|prevtag)$

In [5]:
tagCountDict = {} 
emissionDict = {}
transitionDict = {}

tagtype = 'upos'
data_file = open("UD_Spanish-AnCora/es_ancora-ud-dev.conllu", "r", encoding="utf-8")

# Calculando conteos (pre-probabilidades)
for tokenlist in parse_incr(data_file):
    prevtag = None #Tag previo para el primer token
    for token in tokenlist:

        # C(tag)
        tag = token[tagtype]
        if tag in tagCountDict.keys():
            tagCountDict[tag] += 1
        else:
            tagCountDict[tag] = 1

        # C(word|tag) -> probabilidades emision
        wordtag = token['form'].lower()+'|'+token[tagtype] # (word|tag)
        if wordtag in emissionDict.keys():
            emissionDict[wordtag] = emissionDict[wordtag] + 1
        else:
            emissionDict[wordtag] = 1

        #  C(tag|tag_previo) -> probabilidades transición
        if prevtag is None:
            prevtag = tag
            continue #Salta a la siguiente iteración

        transitiontags = tag+'|'+prevtag
        if transitiontags in transitionDict.keys():
            transitionDict[transitiontags] = transitionDict[transitiontags] + 1
        else:
            transitionDict[transitiontags] = 1
        prevtag = tag
    

#tagCountDict
#emissionDict
#transitionDict

## Entrenamiento del modelo - calculo de probabilidades
* probabilidades de transición:
$$P(tag|prevtag) = \frac{C(prevtag, tag)}{C(prevtag)}$$

* probabilidades de emisión:
 $$P(word|tag) = \frac{C(word|tag)}{C(tag)}$$

In [6]:
transitionProbDict = {} # matriz A
emissionProbDict = {} # matriz B

# transition Probabilities 
for key in transitionDict.keys():
    tag, prevtag = key.split('|')
    if tagCountDict[prevtag]>0:
        transitionProbDict[key] = transitionDict[key]/(tagCountDict[prevtag])
    else:
        print(key)

# emission Probabilities 
for key in emissionDict.keys():
    word, tag = key.split('|')
    if emissionDict[key]>0:
        emissionProbDict[key] = emissionDict[key]/tagCountDict[tag]
    else:
        print(key)

In [7]:
transitionProbDict

{'NOUN|DET': 0.734186313973548,
 'PUNCT|NOUN': 0.21731430357328888,
 'ADP|PUNCT': 0.11054475947754062,
 'ADJ|ADP': 0.020764899108399813,
 'NOUN|ADJ': 0.21553672316384181,
 'ADP|NOUN': 0.38295655797478906,
 'SCONJ|ADP': 0.017832003754106054,
 'VERB|SCONJ': 0.15933098591549297,
 'VERB|VERB': 0.028910802330793368,
 'NUM|VERB': 0.019497982967279247,
 'NOUN|NUM': 0.5508571428571428,
 'NOUN|ADP': 0.26079305490380106,
 'VERB|ADP': 0.05983106522759268,
 'ADP|VERB': 0.27319587628865977,
 'DET|NOUN': 0.014063964996353788,
 'CCONJ|NOUN': 0.049588498801958536,
 'ADJ|CCONJ': 0.08459422283356259,
 'ADP|ADJ': 0.27937853107344635,
 'DET|ADP': 0.38714218676677614,
 'ADJ|NOUN': 0.17033024273361808,
 'PUNCT|ADJ': 0.2480225988700565,
 'VERB|PUNCT': 0.07629818413507486,
 'PRON|DET': 0.018545140885566417,
 'DET|PRON': 0.03709810387469085,
 'PROPN|DET': 0.09272570442783208,
 'PROPN|PROPN': 0.22875494071146246,
 'ADP|PROPN': 0.14303359683794467,
 'PROPN|ADP': 0.1516893477240732,
 'PUNCT|PROPN': 0.367341897233

In [8]:
transitionProbDict['ADJ|ADJ']

0.030225988700564973

In [9]:
emissionProbDict

{'el|DET': 0.24108683151236343,
 'gobernante|NOUN': 0.00020835503698301907,
 ',|PUNCT': 0.45316979929913986,
 'con|ADP': 0.051970905678085405,
 'ganada|ADJ': 0.0002824858757062147,
 'fama|NOUN': 0.00010417751849150954,
 'desde|ADP': 0.008798686062881277,
 'que|SCONJ': 0.6382042253521126,
 'llegó|VERB': 0.0022411474675033617,
 'hace|VERB': 0.009188704616763783,
 '16|NUM': 0.011428571428571429,
 'meses|NOUN': 0.0028127929992707574,
 'al|ADP': 0.04094321914594087,
 'poder|NOUN': 0.0011459527034066049,
 'de|ADP': 0.3748240262787424,
 'explotar|VERB': 0.00044822949350067237,
 'máximo|NOUN': 0.00020835503698301907,
 'su|DET': 0.05031627372052904,
 'oratoria|NOUN': 0.00010417751849150954,
 'y|CCONJ': 0.7771664374140302,
 'acusado|ADJ': 0.000847457627118644,
 'por|ADP': 0.059713749413420926,
 'sus|DET': 0.01998274870615296,
 'detractores|NOUN': 0.0003125325554745286,
 'incontinencia|NOUN': 0.00010417751849150954,
 'verbal|ADJ': 0.0005649717514124294,
 'enmudeció|VERB': 0.00022411474675033618,


In [10]:
emissionProbDict['poderío|NOUN']

0.00010417751849150954

## Guardar parámetros del modelo

In [11]:
import numpy as np
np.save('transitionHMM.npy', transitionProbDict)
np.save('emissionHMM.npy', emissionProbDict)
transitionProbdict = np.load('transitionHMM.npy', allow_pickle='TRUE').item()
transitionProbDict['ADJ|ADJ']

0.030225988700564973

In [12]:
emissionProbDict['poderío|NOUN']

0.00010417751849150954

## Guardar parámetros del modelo

In [13]:
import numpy as np
np.save('transitionHMM.npy', transitionProbDict)
np.save('emissionHMM.npy', emissionProbDict)
transitionProbdict = np.load('transitionHMM.npy', allow_pickle='TRUE').item()
transitionProbDict['ADJ|ADJ']

0.030225988700564973

In [14]:
!jupyter nbconvert --to=python 2_HMM_train.ipynb

[NbConvertApp] Converting notebook 2_HMM_train.ipynb to python
[NbConvertApp] Writing 4500 bytes to 2_HMM_train.py
